## 1. Caricamento dei file contenenti il dataset

In [1]:
import pandas as pd

# Load File and create a new dataframe called 'df'
files = [
    "full_tagged_dataset_2%.csv",
    "full_tagged_dataset_4%.csv",
    "full_tagged_dataset_6%.csv",
    "full_tagged_dataset_8%.csv",
    "full_tagged_dataset_10%.csv"
]

df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

df

,TITLE,NER,difficulty,time,cost,method,INGREDIENTS,DIRECTIONS,total_time,PREPARATION_TIME,NER_clean,VEGAN,NON_VEGAN_INGREDIENTS,VEGETARIAN,NON_VEGETARIAN_INGREDIENTS,total_cost,price_tag,categories,vegan,vegetarian
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",Hard,Slow,Cheap,Bake,"[""16 eggs, beaten"", ""6 -8 evaporated milk, 360...","[""Distribute rolls and cinnamon roll in (4) 2\...",150.0,Very slow (90+ mins),"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",False,"['eggs', 'milk']",True,[],49.2,medium,"['seafood', 'dairy', 'sweetener', 'spice/herb'...",False,False
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",Hard,Average,Cheap,Other,"[""1 (6 ounce) can tuna, drained and flaked"", ""...","[""Put everything in a bowl and mix together un...",15.0,Fast (10-20 mins),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,"['tuna', 'low-fat sour cream', 'bacon bits']",False,"['tuna', 'bacon bits']",52.5,expensive,"['seafood', 'condiment', None, 'meat', 'spice/...",False,False
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",Hard,Quick,Cheap,Bake,"[""1 tablespoon olive oil"", ""1 medium onion , c...","[""Heat oven to 375 degrees."", """", ""In medium s...",70.0,Slow (40-90 mins),"['olive oil', 'onion', 'garlic', 'salt', 'grou...",False,"['ricotta cheese', 'Parmesan cheese', 'egg']",True,[],78.0,rich,"['condiment', 'vegetable', 'vegetable', 'condi...",False,False
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",Hard,Quick,Cheap,Boil,"[""1 teaspoon olive oil"", ""1 cup diced onion"", ...","[""Heat oil in a large skillet over medium-high...",22.0,Medium (20-40 mins),"['olive oil', 'onion', 'red bell pepper', 'gre...",False,['cheddar cheese'],True,[],87.5,rich,"['condiment', 'vegetable', 'vegetable', 'veget...",False,False
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",Hard,Quick,Expensive,Boil,"[""900.0 gs lean ground beef"", ""1 white onion, ...","[""Brown ground beef and onion in a large pot. ...",45.0,Slow (40-90 mins),"['lean ground beef', 'white onion', 'ground bl...",False,"['lean ground beef', 'parmesan cheese', 'sour ...",False,['lean ground beef'],96.3,rich,"['meat', 'vegetable', 'condiment', 'spice/herb...",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223110,Light Wheat Sandwich Bread Recipe,"['warm water', 'active dry yeast', 'honey', 's...",Hard,Slow,Cheap,Bake,"[""1 cup warm water (about 110A F)"", ""2 14 teas...","[""In large mixing bowl, combine warm water and...",290.0,Very slow (90+ mins),"['warm water', 'active dry yeast', 'honey', 's...",False,"['honey', 'butter', 'egg']",True,[],38.4,medium,"['processed', 'processed', 'sweetener', 'condi...",False,False
223111,Low-Fat Corn Pudding,"['corn kernels', 'flour', 'sugar', 'kosher sal...",Hard,Quick,Cheap,Bake,"[""2 cups uncooked corn kernels (from about 4 e...","[""Preheat the oven to 350 degrees. Have ready ...",58.0,Slow (40-90 mins),"['corn kernels', 'flour', 'sugar', 'kosher sal...",False,"['egg substitute', 'butter', 'milk']",True,[],30.4,cheap,"['grain', 'processed', 'sweetener', 'condiment...",False,True
223112,Greek Salata,"['head lettuce', 'radishes', 'green onions', '...",Hard,Average,Expensive,Other,"[""0.5 head lettuce"", ""0.5 bunch radishes, thin...","[""Tear the lettuce into bite size pieces. Toss...",30.0,Medium (20-40 mins),"['head lettuce', 'radishes', 'green onions', '...",False,['Feta cheese'],False,['anchovy'],60.0,expensive,"['vegetable', 'vegetable', 'vegetable', 'fruit...",False,False
223113,Spicy Peanut Soup,"['vegetable oil', 'onion', 'sweet potato', 'ga...",Hard,Quick,Cheap,Boil,"[""1 tablespoon vegetable oil"", ""1 large onion,...","[""In a large saucepan, heat vegetable oil to m...",35.0,Medium (20-40 mins),"['vegetable oil', 'onion', 'sweet potato', 'ga...",False,['chicken broth'],False,['chicken broth'],54.9,expe

## 2. Definizione dei metodi per le conversioni di frazioni e unità di misura e applicazione di essi sul dataframe

In [2]:
from fractions import Fraction


# === UTILITY FUNCTIONS ===

def convert_fractions(text):
    """Convert fractions in the format '1 1/2' or '3/4' to decimals (e.g., '1.5', '0.75')."""
    # Match optional whole number + fraction, or just fraction
    pattern = r'(?:(\d+)\s+)?(\d+)/(\d+)'

    def replacer(match):
        whole = int(match.group(1)) if match.group(1) else 0
        numerator = int(match.group(2))
        denominator = int(match.group(3))
        decimal = whole + Fraction(numerator, denominator)
        return "{:.10g}".format(float(decimal))  # Clean, no trailing zeros

    return re.sub(pattern, replacer, text)


# Fixed conversions
CONVERSIONS = {
    "oz": 30,      # ounce → gram
    "lb": 450,      # pounds → gram
    "pt": 475,      # pint → milliliter
    "qt": 950,      # quart → milliliter
    "inch": 2.5        # inches → centimeter
}

def unit_to_metric(unit):
    """Return the metric equivalent of the given imperial unit."""
    return {
        "oz": "g",
        "lb": "g",
        "pt": "ml",
        "qt": "ml",
        "inch": "cm"
    }[unit]

def convert_units(text):
    """Convert imperial units (oz, lb, pt, qt, inch) to metric units in the text."""
    if pd.isna(text):
        return text

    # Convert oz, lb, pt, qt (form: "4 oz", "2.5 lb", etc.)
    for unit, factor in CONVERSIONS.items():
        pattern = r'(\d+(\.\d+)?)\s*' + unit
        text = re.sub(pattern, lambda m: f"{round(float(m.group(1)) * factor, 1)} {unit_to_metric(unit)}", text)

    # Convert measure in inches in pans
    text = re.sub(r'(\d+)\s*x\s*(\d+)[-\s]*inch', lambda
        m: f"{int(m.group(1)) * CONVERSIONS['inch']:.0f} x {int(m.group(2)) * CONVERSIONS['inch']:.0f} cm", text)

    return text

def convert_fahrenheit_to_celsius(text):
    """Convert Fahrenheit temperatures to Celsius (e.g., 350°F → 177°C)."""
    # Match degrees like 275°, 275 °F, 275\u00b0F (escaped), etc.
    pattern = r'(\d+)\s*(?:°|\\u00b0|degrees)'

    def replacer(match):
        fahrenheit = int(match.group(1))
        celsius = round((fahrenheit - 32) * 5 / 9)
        return f"{celsius}°C"

    return re.sub(pattern, replacer, text)

# righe non utilizzabili perche le colonne non sono presenti nel dataset gia taggato, risolvere come ti avevo gia detto
'''
# === APPLY TRANSFORMATIONS ===

# Convert fractions and units in ingredients and directions
df['INGREDIENTS'] = df['ingredients'].apply(convert_fractions).apply(convert_units)
df['DIRECTIONS'] = df['directions'].apply(convert_fractions).apply(convert_units).apply(convert_fahrenheit_to_celsius)

# Rename title column and drop raw ingredients/directions columns
df = df.rename(columns={"title": "TITLE"}).drop(columns=["ingredients", "directions"])
'''

'\n# === APPLY TRANSFORMATIONS ===\n\n# Convert fractions and units in ingredients and directions\ndf[\'INGREDIENTS\'] = df[\'ingredients\'].apply(convert_fractions).apply(convert_units)\ndf[\'DIRECTIONS\'] = df[\'directions\'].apply(convert_fractions).apply(convert_units).apply(convert_fahrenheit_to_celsius)\n\n# Rename title column and drop raw ingredients/directions columns\ndf = df.rename(columns={"title": "TITLE"}).drop(columns=["ingredients", "directions"])\n'

## 3. Definizione di una funzione in grado di stimare il tempo di preparazione di una ricetta

In [3]:
import re

# Patterns to detect time expressions
TIME_PATTERN = re.compile(
    r'(\d+(?:\.\d+)?)\s*(minute|minutes|min|mins|m|hour|hours|hr|hrs|h|day|days|d)\b',
    re.IGNORECASE
)

RANGE_PATTERN = re.compile(
    r'(\d+)\s*-\s*(\d+)\s*(minute|minutes|min|mins|m|hour|hours|hr|hrs|h)\b',
    re.IGNORECASE
)

# Time estimates for common preparation methods
PREP_ESTIMATES = {
    'bake': 45, 'boil': 20, 'fry': 15, 'grill': 25, 'chill': 120,
    'simmer': 30, 'marinate': 60, 'microwave': 10, 'no-bake': 20,
    'refrigerate': 180, 'freeze': 240
}

# Special cases for recipe types
RECIPE_TYPE_ESTIMATES = {
    'pasta': 10, 'salad': 15, 'cake': 45, 'pie': 60, 'stew': 120,
    'casserole': 60, 'soup': 30, 'cookies': 30, 'bread': 90,
    'fudge': 20, 'candy': 30
}

def convert_to_minutes(qty, unit):
    """Converts time quantity to minutes."""
    qty = float(qty)
    unit = unit.lower()
    if unit.startswith('m'): return qty
    if unit.startswith('h'): return qty * 60
    if unit.startswith('d'): return qty * 1440
    return 0

def estimate_by_recipe_type(title, ingredients):
    """Fallback estimation based on recipe title or ingredients."""
    title = str(title).lower()
    ingredients = str(ingredients).lower()

    # Check for specific recipe types
    for keyword, est in RECIPE_TYPE_ESTIMATES.items():
        if keyword in title:
            return est

    if any(word in ingredients for word in ['raw', 'fresh']):
        return 15
    if 'frozen' in ingredients:
        return 20
    if 'canned' in ingredients:
        return 10

    # Default fallback
    return 30

def clean_instruction_step(step):
    """Normalize range formats and identify special cases."""
    # Replace ranges (e.g., "10-15 minutes" → "15 minutes")
    step = RANGE_PATTERN.sub(lambda m: f"{m.group(2)} {m.group(3)}", step)

    # Handle special keywords
    lowered = step.lower()
    if 'overnight' in lowered:
        return 480
    if 'until set' in lowered or 'until firm' in lowered:
        return 60

    # Sum all time expressions
    return sum(
        convert_to_minutes(qty, unit)
        for qty, unit in TIME_PATTERN.findall(step)
    )

def parse_instructions(instructions):
    """Parses instructions and extracts total estimated time in minutes."""
    total_time = 0

    if not isinstance(instructions, list):
        try:
            instructions = ast.literal_eval(str(instructions))
        except:
            instructions = [instructions]

    for step in instructions:
        if isinstance(step, str):
            total_time += clean_instruction_step(step)

    return total_time

def categorize_time(total_time):
    """Classify total time into labeled categories."""
    if total_time == 0:
        return 'Not specified'
    if total_time < 10:
        return 'Very fast (0-10 mins)'
    if total_time < 20:
        return 'Fast (10-20 mins)'
    if total_time < 40:
        return 'Medium (20-40 mins)'
    if total_time < 90:
        return 'Slow (40-90 mins)'
    return 'Very slow (90+ mins)'

# Calculate total preparation time
df['total_time'] = df['DIRECTIONS'].apply(parse_instructions)

# Estimate time for unspecified recipes
missing = df['total_time'] == 0
df.loc[missing, 'total_time'] = df[missing].apply(
    lambda row: estimate_by_recipe_type(row['TITLE'], row['INGREDIENTS']),
    axis=1
)

# Categorize preparation times
df['PREPARATION_TIME'] = df['total_time'].apply(categorize_time)

df[['TITLE', 'INGREDIENTS', 'DIRECTIONS', 'PREPARATION_TIME']].head(200)

,TITLE,INGREDIENTS,DIRECTIONS,PREPARATION_TIME
0,Western Sizzlin Bread Pudding,"[""16 eggs, beaten"", ""6 -8 evaporated milk, 360...","[""Distribute rolls and cinnamon roll in (4) 2\...",Very slow (90+ mins)
1,Creamy Tuna And Bacon Salad (Reduced Fat),"[""1 (6 ounce) can tuna, drained and flaked"", ""...","[""Put everything in a bowl and mix together un...",Fast (10-20 mins)
2,Spinach And Mushroom Lasagna,"[""1 tablespoon olive oil"", ""1 medium onion , c...","[""Heat oven to 375 degrees."", """", ""In medium s...",Slow (40-90 mins)
3,Three-Bean Tacos,"[""1 teaspoon olive oil"", ""1 cup diced onion"", ...","[""Heat oil in a large skillet over medium-high...",Medium (20-40 mins)
4,Hearty Hamburger Soup,"[""900.0 gs lean ground beef"", ""1 white onion, ...","[""Brown ground beef and onion in a large pot. ...",Slow (40-90 mins)
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"[""450 g sweet potatoes"", ""15 g butter or olive...","[""Peel and cut sweet potatoes into chunks."", ""...",Very slow (90+ mins)
196,Green Cleaner and Bug Repellant in One (Concen...,"[""1 tablespoon essential oil (part)"", ""3 table...","[""Mix all ingredients."", ""Store in a glass jar...",Medium (20-40 mins)
197,Chocolate Chip Pie,"[""1 graham cracker crust"", ""20 large marshmall...","[""Make crust (I buy the prepared ones from the...",Slow (40-90 mins)
198,Poppy Seed Dressing,"[""0.75 c. sugar"", ""2 tsp. dry mustard"", ""2 tsp...","[""Combine first 6 ingredients in a blender con...",Very fast (0-10 mins)


### 4. La cella sottostante contiene una logica per calcolare la categoria di costo di una ricetta, che è stata poi sostituita da un metodo più efficace

In [4]:
'''
# Conversione unità → kg
UNIT_CONVERSION = {
    'cup': 0.24, 'c.': 0.24, 'c': 0.24,
    'teaspoon': 0.005, 'tsp': 0.005, 'tsp.': 0.005,
    'tablespoon': 0.015, 'tbsp': 0.015, 'tbsp.': 0.015,
    'package': 0.2, 'pkg': 0.2, 'pkg.': 0.2,
    'can': 0.4, 'carton': 1.0,
    'g': 0.001, 'g.': 0.001, 'gram': 0.001,
    'kg': 1.0, 'kilogram': 1.0,
    'lb': 0.4536, 'pound': 0.4536,
    'oz': 0.02835, 'ounce': 0.02835,
    'large package': 0.5, 'large pkg.': 0.5, 'large pkg': 0.5
}

# 3. Parsing function (qty, unit, name, grams_in_paren)
def parse_ingredient(ing_str):
    # peso fra parentesi
    m = re.search(r'\(\s*([0-9]+(?:\.[0-9]+)?)\s*g\.?\s*\)', ing_str, re.IGNORECASE)
    grams = float(m.group(1)) if m else None

    # quantity e unit
    patt = r'^\s*([\d\/.\s]+)?\s*([a-zA-Z\.]+)?'
    m2 = re.match(patt, ing_str)
    qty_raw = m2.group(1) if m2 else None
    try:
        qty = eval(qty_raw.replace(' ', '+')) if qty_raw else 1.0
    except:
        qty = 1.0
    unit = (m2.group(2) or '').strip('.').lower() if m2 else ''

    # name pulito
    name = re.sub(r'\(.*?\)|optional', '', ing_str, flags=re.IGNORECASE)
    # rimuovo quantità/unità
    name = re.sub(r'^[\d\/.\s]+\s*[a-zA-Z\.]*', '', name).strip().lower()
    return qty, unit, name, grams

# 4. Calcolo posizionale con fallback substring e token
def calculate_recipe_cost_positional(ingredients_list, ner_list):
    total = 0.0
    missing = []

    for ing_str, ner_item in zip(ingredients_list, ner_list):
        key = ner_item.lower()
        qty, unit, name, grams = parse_ingredient(ing_str)

        # scorporo se passo i grams
        if grams is not None:
            kg = (grams / 1000) * qty
        elif unit == '':  # a pezzo
            # prendo prezzo direct key
            price = price_dict.get(key)
            # fallback substring/token
            if price is None:
                # substring match
                for k, v in price_dict.items():
                    if k in key or key in k:
                        price = v
                        break
                # token match
                if price is None:
                    for token in key.split():
                        if token in price_dict:
                            price = price_dict[token]
                            break
            if price:
                total += qty * price
            else:
                missing.append(key)
            continue
        else:
            conv = UNIT_CONVERSION.get(unit, 0.0)
            if conv == 0:
                missing.append(key)
                continue
            kg = qty * conv

        # prezzo per kg
        price = price_dict.get(key)
        if price is None:
            for k, v in price_dict.items():
                if k in key or key in k:
                    price = v
                    break
        if price is None:
            missing.append(key)
            continue

        total += kg * price

    return round(total, 2), missing

# 5. Applica al DataFrame
    df['INGREDIENTS'] = df['INGREDIENTS'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['NER_clean']  = df['NER_clean'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def compute_cost_row(row):
     cost, _ = calculate_recipe_cost_positional(row['INGREDIENTS'], row['NER_clean'])
     return cost

df['total_cost'] = df.apply(compute_cost_row, axis=1)

df['CATEGORY_COST'] = df['total_cost'].apply(categorize_cost)

# Mostra risultati
df[['INGREDIENTS', 'NER', 'CATEGORY_COST']].head(200)'''

"\n# Conversione unità → kg\nUNIT_CONVERSION = {\n    'cup': 0.24, 'c.': 0.24, 'c': 0.24,\n    'teaspoon': 0.005, 'tsp': 0.005, 'tsp.': 0.005,\n    'tablespoon': 0.015, 'tbsp': 0.015, 'tbsp.': 0.015,\n    'package': 0.2, 'pkg': 0.2, 'pkg.': 0.2,\n    'can': 0.4, 'carton': 1.0,\n    'g': 0.001, 'g.': 0.001, 'gram': 0.001,\n    'kg': 1.0, 'kilogram': 1.0,\n    'lb': 0.4536, 'pound': 0.4536,\n    'oz': 0.02835, 'ounce': 0.02835,\n    'large package': 0.5, 'large pkg.': 0.5, 'large pkg': 0.5\n}\n\n# 3. Parsing function (qty, unit, name, grams_in_paren)\ndef parse_ingredient(ing_str):\n    # peso fra parentesi\n    m = re.search(r'\\(\\s*([0-9]+(?:\\.[0-9]+)?)\\s*g\\.?\\s*\\)', ing_str, re.IGNORECASE)\n    grams = float(m.group(1)) if m else None\n\n    # quantity e unit\n    patt = r'^\\s*([\\d\\/.\\s]+)?\\s*([a-zA-Z\\.]+)?'\n    m2 = re.match(patt, ing_str)\n    qty_raw = m2.group(1) if m2 else None\n    try:\n        qty = eval(qty_raw.replace(' ', '+')) if qty_raw else 1.0\n    except:\

## 5. Definizione di una funzione in grado di determinare se una ricetta è vegana oppure no

In [5]:
import ast
import re

# Cleaning the NER column from special symbols
df['NER_clean'] = df['NER'].apply(ast.literal_eval)

# List containing keywords
NON_VEGAN_KEYWORDS = {
    # meat
    'meat', 'red meat', 'white meat', 'poultry', 'veal', 'lamb', 'duck',
    'goose', 'chicken', 'beef', 'pork', 'bacon', 'ham', 'prosciutto',
    'sausage', 'salami', 'hot dog', 'pepperoni', 'mortadella', 'chorizo',
    'ground beef', 'minced meat', 'steak', 'ribs', 'brisket', 'lardons',

    # fish or seafood
    'fish', 'anchovy', 'anchovies', 'tuna', 'salmon', 'sardines', 'mackerel',
    'cod', 'haddock', 'tilapia', 'trout', 'snapper', 'catfish', 'bass',
    'shellfish', 'shrimp', 'prawns', 'crab', 'lobster', 'scampi', 'clam',
    'mussels', 'oysters', 'scallops', 'caviar', 'roe',

    # dairy
    'milk', 'whole milk', 'skim milk', 'cheese', 'butter', 'cream',
    'heavy cream', 'sour cream', 'whipped cream', 'yogurt', 'ghee',
    'lactose', 'casein', 'whey', 'buttermilk', 'condensed milk',
    'evaporated milk', 'parmesan', 'mozzarella', 'cheddar',
    'shredded cheese', 'blue cheese', 'goat cheese', 'cream cheese',
    'cottage cheese', 'ricotta', 'cocoa butter',

    # eggs
    'egg', 'eggs', 'egg yolk', 'egg white', 'albumen', 'mayonnaise',

    # bees
    'honey', 'royal jelly', 'propolis', 'bee pollen',

    # animal derivatives
    'gelatin', 'gelatina', 'collagen', 'isinglass', 'lard', 'suet',
    'carmine', 'shellac', 'pepsin', 'bone broth', 'bone marrow',
    'broth (meat)', 'stock (meat)', 'chicken broth', 'beef broth',
    'animal fat'
}

VEGAN_EXCEPTIONS = {
    'milk': {'soy', 'almond', 'oat', 'rice', 'coconut', 'cashew', 'hazelnut',
             'hemp', 'pea', 'flax', 'macadamia', 'barley', 'plant-based milk'},
    'cheese': {'vegan cheese', 'nutritional yeast', 'cashew cheese', 'tofu cheese',
               'almond cheese', 'plant-based cheese'},
    'meat': {'soy', 'seitan', 'tofu', 'tempeh', 'jackfruit', 'plant-based', 'textured soy protein', 'mushroom-based'},
    'butter': {'vegan butter', 'plant butter', 'peanut butter', 'almond butter',
               'sunflower butter', 'coconut butter', 'cashew butter'},
    'cream': {'coconut cream', 'soy cream', 'oat cream', 'vegan cream', 'cashew cream'},
    'bacon': {'vegan bacon', 'tempeh bacon', 'coconut bacon', 'mushroom bacon'},
    'egg': {'flaxseed', 'chia', 'aquafaba', 'vegan egg', 'plant-based egg', 'apple sauce'}
}

VEGAN_MODIFIERS = {
    'vegan', '100% vegan', 'plant-based', 'dairy-free', 'egg-free',
    'no milk', 'no eggs', 'cruelty-free', 'without animal derivatives',
    '100% plant-based', 'meat-free', 'no animal products'
}

# --- UTILITY (Used in vegan, vegetarian, lactose free and gluten free)
def parse_ingredients(ingredients_input):
    """Always returning a parsed list of ingredients"""
    if isinstance(ingredients_input, list):
        return ingredients_input
    if isinstance(ingredients_input, str) and ingredients_input.strip():
        try:
            return ast.literal_eval(ingredients_input)
        except:
            return [
                x.strip().strip('"').strip("'")
                for x in re.split(r',(?![^[]*\])', ingredients_input.strip('[]'))
                if x.strip()
            ]
    return []

def contains_vegan_exception(ingredient, keyword):
    return any(re.search(rf'\b{re.escape(ex)}\b', ingredient, re.IGNORECASE)
               for ex in VEGAN_EXCEPTIONS.get(keyword, set()))

# --- Classification
def classify_vegan(ingredients_input):
    """Returning True or False based on the ingredients"""
    ingredients = parse_ingredients(ingredients_input)
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        if any(re.search(rf'\b{re.escape(mod)}\b', ing_lower) for mod in VEGAN_MODIFIERS):
            continue
        for keyword in NON_VEGAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegan_exception(ing_lower, keyword):
                    return False
    return True

def get_non_vegan_ingredients(ingredients_input):
    """Returning list of ingredients without vegan exceptions"""
    ingredients = parse_ingredients(ingredients_input)
    non_vegan = []
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        for keyword in NON_VEGAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegan_exception(ing_lower, keyword):
                    non_vegan.append(ing)
                break
    return non_vegan

# --- Apply
df['VEGAN'] = df['NER_clean'].apply(classify_vegan)
df['NON_VEGAN_INGREDIENTS'] = df['NER_clean'].apply(get_non_vegan_ingredients)

# --- Final validation
anti_patterns = re.compile(
    r'\b(not vegan|meat|steak|fish|cheese|egg|eggs|beef|chicken|pork|bacon|cream)\b',
    re.IGNORECASE
)

df['VEGAN'] = df.apply(
    lambda row: False if row['VEGAN'] and isinstance(row['TITLE'], str) and anti_patterns.search(row['TITLE']) else row['VEGAN'],
    axis=1
)

# View results
df[['TITLE', 'NER', 'VEGAN', 'NON_VEGAN_INGREDIENTS']].head(200)

,TITLE,NER,VEGAN,NON_VEGAN_INGREDIENTS
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",False,"[eggs, milk]"
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,"[tuna, mayonnaise, low-fat sour cream, bacon b..."
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",False,"[ricotta cheese, Parmesan cheese, egg, mozzare..."
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",False,[cheddar cheese]
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",False,"[lean ground beef, parmesan cheese, sour cream]"
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"['sweet potatoes', 'butter', 'salt', 'flour', ...",False,[butter]
196,Green Cleaner and Bug Repellant in One (Concen...,"['essential oil', 'vodka', 'liquid soap']",True,[]
197,Chocolate Chip Pie,"['graham cracker crust', 'marshmallows', 'milk...",False,"[milk, whipping cream]"
198,Poppy Seed Dressing,"['sugar', 'dry mustard', 'salt', 'vinegar', 'o...",True,[]


## 6. Definizione di una funzione in grado di determinare se una ricetta è vegetariana oppure no

In [6]:
# List of non vegetarian ingredients
NON_VEGETARIAN_KEYWORDS = {
    'meat', 'red meat', 'white meat', 'poultry', 'veal', 'lamb', 'duck',
    'goose', 'chicken', 'beef', 'pork', 'bacon', 'ham', 'prosciutto',
    'sausage', 'salami', 'hot dog', 'pepperoni', 'mortadella', 'chorizo',
    'ground beef', 'minced meat', 'steak', 'ribs', 'brisket', 'lardons',
    'fish', 'anchovy', 'anchovies', 'tuna', 'salmon', 'sardines', 'mackerel',
    'cod', 'haddock', 'tilapia', 'trout', 'snapper', 'catfish', 'bass',
    'shellfish', 'shrimp', 'prawns', 'crab', 'lobster', 'scampi', 'clam',
    'mussels', 'oysters', 'scallops', 'caviar', 'roe',
    'gelatin', 'lard', 'collagen', 'isinglass', 'pepsin', 'suet', 'bone broth',
    'bone marrow', 'stock (meat)', 'bouillon (meat)', 'chicken broth', 'beef broth'
}

# Exceptions for vegetarian subs
VEGETARIAN_EXCEPTIONS = {
    'meat': {'soy', 'soy protein', 'textured vegetable protein', 'tvp', 'seitan', 'tofu',
             'tempeh', 'jackfruit', 'lentils', 'chickpeas', 'plant-based', 'mycoprotein', 'quorn'},
    'bacon': {'vegan bacon', 'tempeh bacon', 'coconut bacon', 'rice paper bacon'},
    'gelatin': {'agar', 'pectin', 'carrageenan', 'guar gum', 'xanthan gum'},
    'fish': {'banana blossom', 'tofu', 'plant-based', 'seaweed', 'nori', 'smoked carrot'}
}

VEGETARIAN_MODIFIERS = {
    'vegetarian', 'veggie', 'plant-based', 'meatless', 'no meat',
    'without meat', 'cruelty-free', 'ovo-vegetarian', 'lacto-vegetarian',
    '100% vegetarian', 'meat-free', 'green option'
}

def contains_vegetarian_exception(ingredient, keyword):
    exceptions = VEGETARIAN_EXCEPTIONS.get(keyword, set())
    return any(re.search(rf'\b{re.escape(ex)}\b', ingredient, re.IGNORECASE) for ex in exceptions)

# --- Classification
def classify_vegetarian(ingredients_input):
    """Returning True or False based on the ingredients"""
    ingredients = parse_ingredients(ingredients_input) # using the function 'parse_ingredients' defined in the previous cell
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        if any(re.search(rf'\b{re.escape(mod)}\b', ing_lower) for mod in VEGETARIAN_MODIFIERS):
            continue
        for keyword in NON_VEGETARIAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegetarian_exception(ing_lower, keyword):
                    return False
    return True

def get_non_vegetarian_ingredients(ingredients_input):
    """Returning list of ingredients without vegetarian exceptions"""
    ingredients = parse_ingredients(ingredients_input)
    non_vegetarian = []
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        for keyword in NON_VEGETARIAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegetarian_exception(ing_lower, keyword):
                    non_vegetarian.append(ing)
                break
    return non_vegetarian

# Apply to dataframe
df['VEGETARIAN'] = df['NER_clean'].apply(classify_vegetarian)
df['NON_VEGETARIAN_INGREDIENTS'] = df['NER_clean'].apply(get_non_vegetarian_ingredients)

anti_patterns = re.compile(
    r'\b(not vegetarian|not veg|meat|steak|fish|beef|chicken|pork|bacon)\b',
    re.IGNORECASE
)

df['VEGETARIAN'] = df.apply(
    lambda row: False if row['VEGETARIAN'] and isinstance(row['TITLE'], str) and anti_patterns.search(row['TITLE']) else row['VEGETARIAN'],
    axis=1
)

# Show results
df[['TITLE', 'NER', 'VEGETARIAN', 'NON_VEGETARIAN_INGREDIENTS']].head(200)

,TITLE,NER,VEGETARIAN,NON_VEGETARIAN_INGREDIENTS
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",True,[]
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,"[tuna, bacon bits]"
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",True,[]
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",True,[]
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",False,[lean ground beef]
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"['sweet potatoes', 'butter', 'salt', 'flour', ...",True,[]
196,Green Cleaner and Bug Repellant in One (Concen...,"['essential oil', 'vodka', 'liquid soap']",True,[]
197,Chocolate Chip Pie,"['graham cracker crust', 'marshmallows', 'milk...",True,[]
198,Poppy Seed Dressing,"['sugar', 'dry mustard', 'salt', 'vinegar', 'o...",True,[]


## 7. Definizione di una funzione in grado di stimare il metodo di preparazione della ricetta

In [7]:
# Extended map for methods and keywords
cooking_methods = {
    'Baked': [
        'bake', 'baked', 'baking', 'preheat oven', 'pre-heated oven', 'ovenproof',
        'place in oven', 'roast in oven', 'bake at', 'oven-bake', 'put in oven', 'oven cook'
    ],
    'Boiled': [
        'boil', 'boiled', 'boiling', 'bring to a boil', 'simmer', 'parboil', 'blanch',
        'blanched', 'boiled in', 'simmered', 'cook in boiling water', 'gentle boil', 'rapid boil'
    ],
    'Broil': [
        'broil', 'broiled', 'broiling', 'under the broiler', 'place under broiler', 'broiler setting'
    ],
    'Fried': [
        'fry', 'fried', 'frying', 'deep fry', 'deep-fry', 'pan fry', 'pan-fried', 'shallow fry',
        'stir-fry', 'sauté', 'sautée', 'sautéed', 'sauted', 'sautéing', 'sear', 'seared',
        'brown in a pan', 'browned', 'cook in oil', 'crisp in pan', 'fry in skillet', 'flash fry'
    ],
    'Grilled': [
        'grill', 'grilled', 'grilling', 'grill pan', 'on the grill', 'barbecue', 'bbq',
        'cook over grill', 'char-grill', 'chargrilled', 'griddle pan', 'grilled over flame'
    ],
    'Roasted': [
        'roast', 'roasted', 'roasting', 'oven-roasted', 'slow roast', 'roast in oven',
        'roast at', 'roasted until golden', 'dry-roast', 'dry roast'
    ],
    'Slow Cooking': [
        'slow cook', 'slow-cooked', 'slow-cook', 'slow cooking', 'crockpot', 'crock pot',
        'slow cooker', 'cook on low for', 'long cook time', 'set to low', 'set to high and cook for hours'
    ],
    'Steamed': [
        'steam', 'steamed', 'steaming', 'in steamer', 'steamer basket', 'double boiler',
        'steam until tender', 'cook with steam', 'steam over water', 'steam for minutes'
    ],
    'Pressure Cooked': [
        'pressure cook', 'pressure cooked', 'pressure cooker', 'instant pot',
        'cooked under pressure', 'cook under pressure', 'high pressure', 'manual pressure cook'
    ],
    'Microwaved': [
        'microwave', 'microwaved', 'microwaving', 'cook in microwave', 'microwave on high',
        'microwave-safe bowl', 'microwave for minutes'
    ],
    'Sous Vide': [
        'sous vide', 'sous-vide', 'precision cooker', 'immersion circulator',
        'vacuum sealed and cooked in water bath', 'vacuum seal and cook at temperature',
        'cook sous vide'
    ],
    'Toasted': [
        'toast', 'toasted', 'toasting', 'in toaster', 'toasted until golden',
        'under grill until golden', 'toast under broiler', 'toast on skillet'
    ],
    'Blended': [
        'blend', 'blended', 'blending', 'blender', 'puree', 'pureed', 'pureeing',
        'food processor', 'process until smooth', 'blend until creamy', 'blend until combined',
        'whizz in blender', 'pulse until smooth'
    ],
    'Raw': [
        'serve raw', 'uncooked', 'no cook', 'eat raw', 'fresh only', 'as is', 'ready to eat',
        'no heating required', 'chilled and served', 'consume without cooking', 'fresh and raw'
    ]
}

def detect_cooking_method(instruction_list):
    '''A method to detect cooking method'''
    if isinstance(instruction_list, str):
        try:
            steps = ast.literal_eval(instruction_list.lower())
        except:
            steps = [instruction_list.lower()]
    else:
        steps = [str(instruction).lower() for instruction in instruction_list]

    for method, keywords in cooking_methods.items():
        if any(any(kw in step for kw in keywords) for step in steps):
            return method
    return 'Other'

df['COOKING_METHOD'] = df['DIRECTIONS'].apply(detect_cooking_method)

df[['TITLE', 'DIRECTIONS', 'COOKING_METHOD']].head(200)

,TITLE,DIRECTIONS,COOKING_METHOD
0,Western Sizzlin Bread Pudding,"[""Distribute rolls and cinnamon roll in (4) 2\...",Baked
1,Creamy Tuna And Bacon Salad (Reduced Fat),"[""Put everything in a bowl and mix together un...",Other
2,Spinach And Mushroom Lasagna,"[""Heat oven to 375 degrees."", """", ""In medium s...",Baked
3,Three-Bean Tacos,"[""Heat oil in a large skillet over medium-high...",Boiled
4,Hearty Hamburger Soup,"[""Brown ground beef and onion in a large pot. ...",Boiled
...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"[""Peel and cut sweet potatoes into chunks."", ""...",Baked
196,Green Cleaner and Bug Repellant in One (Concen...,"[""Mix all ingredients."", ""Store in a glass jar...",Other
197,Chocolate Chip Pie,"[""Make crust (I buy the prepared ones from the...",Boiled
198,Poppy Seed Dressing,"[""Combine first 6 ingredients in a blender con...",Blended


## 8. Definizione di una funzione in grado di stimare la difficoltà di una ricetta

In [8]:
import ast

def estimate_difficulty(row):
    try:
        ingredients = ast.literal_eval(row['INGREDIENTS'])
        instructions = ast.literal_eval(row['DIRECTIONS'])
    except Exception:
        return 'Unknown'

    num_ingredients = len(ingredients)
    num_steps = len([s for s in instructions if s.strip()])
    instruction_text = ' '.join(instructions).lower()

    mild_keywords = [
        'mix', 'combine', 'stir', 'spread', 'pour', 'season',
        'slice', 'chop', 'cut', 'heat', 'grease', 'add'
    ]
    medium_keywords = [
        'simmer', 'boil', 'bake', 'grill', 'saute', 'blend',
        'fry', 'knead', 'rest', 'whisk', 'preheat', 'chill',
        'melt', 'marinate', 'brown'
    ]
    hard_keywords = [
        'sous vide', 'caramelize', 'double boiler', 'bain-marie',
        'julienne', 'poach', 'deglaze', 'layer', 'water bath',
        'refrigerate overnight', 'fold', 'temper', 'pass through sieve',
        'glaze', 'rotate pans', 'cool completely', 'remove from mold'
    ]

    def keyword_score(keywords, text, max_points=3):
        score = 0
        for kw in keywords:
            matches = len(re.findall(rf'\b{re.escape(kw)}\b', text))
            score += matches
        return min(score, max_points)

    mild_score = keyword_score(mild_keywords, instruction_text)
    medium_score = keyword_score(medium_keywords, instruction_text)
    hard_score = keyword_score(hard_keywords, instruction_text, max_points=4)

    bonus = 0
    if 'overnight' in instruction_text or 'refrigerate overnight' in instruction_text:
        bonus += 3
    if 'cool completely' in instruction_text or 'let cool' in instruction_text:
        bonus += 2
    if 'food processor' in instruction_text:
        bonus += 1
    if 'strain' in instruction_text or 'sieve' in instruction_text:
        bonus += 1
    if 'double boiler' in instruction_text or 'bain-marie' in instruction_text:
        bonus += 2
    if 'fold' in instruction_text:
        bonus += 1

    complex_ingredients = ['gelatin', 'saffron', 'truffle', 'wasabi', 'caviar']
    complex_ingredients_count = sum(1 for ingr in ingredients if any(ci in ingr.lower() for ci in complex_ingredients))
    bonus += complex_ingredients_count * 1.5

    score = 0
    score += num_ingredients * 0.25
    score += num_steps * 1.5
    score += mild_score * 1
    score += medium_score * 2
    score += hard_score * 3
    score += bonus

    # thresholds for the 3 classes (da rivedere)
    if score <= 10:
        return 'Easy'
    elif score <= 22:
        return 'Medium'
    else:
        return 'Hard'

# Apply
df['DIFFICULTY'] = df.apply(estimate_difficulty, axis=1)

df[['TITLE', 'DIRECTIONS', 'DIFFICULTY']].head(200)

,TITLE,DIRECTIONS,DIFFICULTY
0,Western Sizzlin Bread Pudding,"[""Distribute rolls and cinnamon roll in (4) 2\...",Medium
1,Creamy Tuna And Bacon Salad (Reduced Fat),"[""Put everything in a bowl and mix together un...",Easy
2,Spinach And Mushroom Lasagna,"[""Heat oven to 375 degrees."", """", ""In medium s...",Hard
3,Three-Bean Tacos,"[""Heat oil in a large skillet over medium-high...",Hard
4,Hearty Hamburger Soup,"[""Brown ground beef and onion in a large pot. ...",Hard
...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"[""Peel and cut sweet potatoes into chunks."", ""...",Hard
196,Green Cleaner and Bug Repellant in One (Concen...,"[""Mix all ingredients."", ""Store in a glass jar...",Easy
197,Chocolate Chip Pie,"[""Make crust (I buy the prepared ones from the...",Hard
198,Poppy Seed Dressing,"[""Combine first 6 ingredients in a blender con...",Medium


## 9. Definizione di una funzione in grado di rilevare se la ricetta contiene lattosio

In [9]:
# List of lactose ingredients
LACTOSE_KEYWORDS = {
    # milks and derivatives
    'milk', 'whole milk', 'skim milk', 'raw milk', 'lactose', 'lactose monohydrate',
    'cream', 'heavy cream', 'double cream', 'single cream', 'whipping cream',
    'butter', 'ghee', 'clarified butter',

    # cheese
    'cheese', 'cream cheese', 'soft cheese', 'hard cheese',
    'ricotta', 'mozzarella', 'parmesan', 'pecorino', 'feta', 'brie', 'camembert',
    'blue cheese', 'goat cheese', 'cheddar', 'colby', 'gouda', 'gruyere',
    'swiss cheese', 'romano cheese', 'mascarpone', 'paneer', 'provolone',

    # Yogurt and similar
    'yogurt', 'greek yogurt', 'kefir', 'curd',

    # desserts and industrials
    'custard', 'ice cream', 'gelato', 'milkshake', 'pudding', 'white chocolate',

    # technical ingredients
    'whey', 'whey protein', 'whey powder', 'casein', 'caseinate', 'milk solids',
    'milk powder', 'dried milk', 'nonfat milk', 'low-fat milk', 'milk protein',
    'milkfat', 'buttermilk', 'evaporated milk', 'condensed milk', 'recombined milk'
}

# Exceptions for lactose free variants
LACTOSE_EXCEPTIONS = {
    'milk': {
        'almond milk', 'soy milk', 'oat milk', 'coconut milk', 'rice milk',
        'lactose-free milk', 'hemp milk', 'pea milk', 'cashew milk',
        'hazelnut milk', 'macadamia milk', 'flax milk'
    },
    'cheese': {
        'vegan cheese', 'plant-based cheese', 'nut cheese', 'cashew cheese',
        'almond cheese', 'tofu cheese', 'soy cheese'
    },
    'butter': {
        'vegan butter', 'plant-based butter', 'margarine', 'oil spread'
    },
    'cream': {
        'coconut cream', 'soy cream', 'oat cream', 'cashew cream', 'almond cream'
    },
    'yogurt': {
        'coconut yogurt', 'soy yogurt', 'almond yogurt', 'oat yogurt',
        'lactose-free yogurt', 'rice yogurt', 'cashew yogurt'
    }
}

def contains_lactose_exception(ingredient, keyword):
    exceptions = LACTOSE_EXCEPTIONS.get(keyword, set())
    return any(re.search(rf'\b{re.escape(ex)}\b', ingredient, re.IGNORECASE) for ex in exceptions)

def classify_lactose_free(ingredients_input):
    ingredients = parse_ingredients(ingredients_input)
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        for keyword in LACTOSE_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_lactose_exception(ing_lower, keyword):
                    return False
    return True

def get_lactose_ingredients(ingredients_input):
    ingredients = parse_ingredients(ingredients_input)
    lactose = []
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        for keyword in LACTOSE_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_lactose_exception(ing_lower, keyword):
                    lactose.append(ing)
                break
    return lactose

# Apply to dataframe
df['LACTOSE_FREE'] = df['NER_clean'].apply(classify_lactose_free)
df['LACTOSE_INGREDIENTS'] = df['NER_clean'].apply(get_lactose_ingredients)

# Show results
df[['TITLE', 'NER', 'LACTOSE_FREE', 'LACTOSE_INGREDIENTS']].head(200)

,TITLE,NER,LACTOSE_FREE,LACTOSE_INGREDIENTS
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",False,[milk]
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,[low-fat sour cream]
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",False,"[ricotta cheese, Parmesan cheese, mozzarella]"
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",False,[cheddar cheese]
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",False,"[parmesan cheese, sour cream]"
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"['sweet potatoes', 'butter', 'salt', 'flour', ...",False,[butter]
196,Green Cleaner and Bug Repellant in One (Concen...,"['essential oil', 'vodka', 'liquid soap']",True,[]
197,Chocolate Chip Pie,"['graham cracker crust', 'marshmallows', 'milk...",False,"[milk, whipping cream]"
198,Poppy Seed Dressing,"['sugar', 'dry mustard', 'salt', 'vinegar', 'o...",True,[]


## 10. Definizione di una funzione in grado di rilevare se la ricetta contiene glutine

In [10]:
# List of gluten ingredients
GLUTEN_KEYWORDS = {
    # cereals containing gluten
    'wheat', 'barley', 'rye', 'spelt', 'kamut', 'triticale', 'farro', 'einkorn', 'emmer',

    # derivatives
    'semolina', 'durum', 'graham flour', 'matzo', 'gluten', 'vital wheat gluten',
    'hydrolyzed wheat protein', 'wheat starch', 'modified wheat starch',
    'malted barley', 'malted milk', 'malt', 'malt extract', 'malt syrup', 'malt vinegar',

    # industrial ingredients
    'brewer’s yeast', 'brewer yeast', 'yeast extract', 'textured vegetable protein (wheat)',
    'vegetable starch (wheat)', 'vegetable gum (wheat)', 'natural flavor (wheat)',
    'seasoned flour', 'spice mix (with wheat)',

    # common products
    'seitan', 'bran', 'bread', 'pasta', 'noodles', 'crackers', 'breadcrumbs', 'biscuit',
    'cake flour', 'pastry flour', 'self-rising flour', 'bagel', 'croissant', 'brioche',
    'pancake mix', 'waffle mix', 'muffin mix', 'pie crust', 'pizza crust', 'tortilla (wheat)',

    # transformed products
    'beer', 'ale', 'lager', 'stuffing mix', 'gravy mix', 'roux', 'soups (with pasta/noodles)',
    'sauces (with flour)', 'soy sauce', 'teriyaki sauce', 'tempura', 'coating mix', 'breading',

    # desserts and snacks
    'ladyfinger', 'graham cracker', 'pretzel', 'cereal bar', 'energy bar (with gluten grains)'
}

# Exceptions for gluten free
GLUTEN_EXCEPTIONS = {
    'wheat': {'wheatgrass', 'wheat-free tamari'},
    'malt': {'maltodextrin (from corn)', 'maltodextrin from corn', 'rice malt', 'corn malt'},
    'starch': {
        'corn starch', 'potato starch', 'tapioca starch', 'arrowroot starch',
        'sago starch', 'rice starch'
    },
    'flour': {
        'almond flour', 'coconut flour', 'chickpea flour', 'rice flour', 'corn flour',
        'potato flour', 'soy flour', 'buckwheat flour', 'tapioca flour', 'quinoa flour',
        'sorghum flour', 'teff flour', 'millet flour', 'amaranth flour', 'cassava flour',
        'oat flour (gluten-free)', 'oat flour gluten-free', 'lentil flour', 'pea flour'
    },
    'bran': {
        'rice bran', 'corn bran', 'oat bran (gluten-free)', 'oat bran gluten-free'
    },
    'bread': {'gluten-free bread'},
    'pasta': {'gluten-free pasta', 'rice pasta', 'quinoa pasta'},
    'noodles': {'rice noodles', 'glass noodles', 'sweet potato noodles', 'mung bean noodles'},
    'beer': {'gluten-free beer', 'sorghum beer'},
    'cracker': {'gluten-free crackers'},
    'breadcrumbs': {'gluten-free breadcrumbs', 'crushed cornflakes (GF)'},
    'biscuit': {'gluten-free biscuit'},
    'soy sauce': {'tamari (gluten-free)', 'gluten-free soy sauce'},
    'gravy mix': {'gluten-free gravy'},
    'stuffing mix': {'gluten-free stuffing'},
    'pizza crust': {'gluten-free pizza crust'},
    'cereal bar': {'gluten-free cereal bar'},
    'breading': {'gluten-free breading'},
    'coating mix': {'gluten-free coating'}
}

def contains_gluten_exception(ingredient, keyword):
    exceptions = GLUTEN_EXCEPTIONS.get(keyword, set())
    return any(re.search(rf'\b{re.escape(ex)}\b', ingredient, re.IGNORECASE) for ex in exceptions)

def classify_gluten_free(ingredients_input):
    ingredients = parse_ingredients(ingredients_input)
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        for keyword in GLUTEN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_gluten_exception(ing_lower, keyword):
                    return False
    return True

def get_gluten_ingredients(ingredients_input):
    ingredients = parse_ingredients(ingredients_input)
    gluten = []
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        for keyword in GLUTEN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_gluten_exception(ing_lower, keyword):
                    gluten.append(ing)
                break
    return gluten

# Apply to dataframe
df['GLUTEN_FREE'] = df['NER_clean'].apply(classify_gluten_free)
df['GLUTEN_INGREDIENTS'] = df['NER_clean'].apply(get_gluten_ingredients)

# Show results
df[['TITLE', 'NER', 'GLUTEN_FREE', 'GLUTEN_INGREDIENTS']].head(200)

,TITLE,NER,GLUTEN_FREE,GLUTEN_INGREDIENTS
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",True,[]
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",True,[]
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",False,[lasagna noodles]
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",True,[]
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",False,[soy sauce]
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"['sweet potatoes', 'butter', 'salt', 'flour', ...",False,[bread flour]
196,Green Cleaner and Bug Repellant in One (Concen...,"['essential oil', 'vodka', 'liquid soap']",True,[]
197,Chocolate Chip Pie,"['graham cracker crust', 'marshmallows', 'milk...",False,[graham cracker crust]
198,Poppy Seed Dressing,"['sugar', 'dry mustard', 'salt', 'vinegar', 'o...",True,[]


## La cella sottostante può essere eseguita solo su computer potenti, e il suo scopo principale è quello di generare la categoria di prezzo della ricetta, e usa un diverso approccio anche per definire altre categorie a cui appartengono le ricette

Qui inizia la magia:

In [11]:
import numpy as np
import torch
from transformers import pipeline
import re
from tqdm import tqdm

gpu = torch.cuda.get_device_name(0)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
print(f"Current device: {torch.cuda.current_device()}")
print(f"Device name: {gpu}")

# Eseguire questa cella solo se runnata dal Barone
if gpu == "NVIDIA GeForce RTX 3060 Ti":
    classifier = pipeline(
        task="zero-shot-classification",
        model="MoritzLaurer/deberta-v3-base-zeroshot-v1.1-all-33",
        device=0,
        torch_dtype=torch.float16,
        model_kwargs={"cache_dir": "./cache"},
        batch_size=32,  # Optimal for RTX 3060 Ti
        framework="pt"
    )
    
    # %% Data Loading
    df['NER'] = df['NER'].apply(eval)  # Convert string lists to actual lists
    
    # %% Food Categories & Prices (EUR/kg)
    CATEGORIES = [
        "dairy", "meat", "seafood", "grain", "vegetable",
        "fruit", "spice/herb", "processed", "sweetener",
        "condiment", "legume", "oil/fat"
    ]
    
    MEDIAN_PRICES = {
        "dairy": 3.50,       # Milk, cheese
        "meat": 7.50,        # Chicken, beef
        "seafood": 12.00,    # Fish, shrimp
        "grain": 2.20,       # Flour, rice
        "vegetable": 1.80,   # Onions, garlic
        "fruit": 2.50,       # Tomatoes, bananas
        "spice/herb": 18.00, # Vanilla, cinnamon
        "processed": 4.50,   # Pasta, canned goods
        "sweetener": 2.20,   # Sugar
        "condiment": 5.00,   # Mayo, dressings
        "legume": 3.00,      # Beans, lentils
        "oil/fat": 8.00      # Olive oil
    }
    
    # %% Ingredient Cleaning (Fixed)
    def clean_ingredient(ingredient: str) -> str:
        """Conservative cleaning preserving ingredient names"""
        # Remove quantities (e.g., "200g", "1/2 cup")
        cleaned = re.sub(r'\b\d+[\d/\.]*\s*[a-z]*\b', '', ingredient, flags=re.IGNORECASE)
        # Remove special chars except spaces
        cleaned = re.sub(r'[^\w\s]', '', cleaned).strip().lower()
        return cleaned if cleaned else "unknown"
    
    # %% Batch Classification (GPU-optimized)
    classification_cache = {}
    
    def batch_classify(ingredients: list, batch_size: int = 16) -> dict:  # Reduced batch size
        unique_ingredients = list(set(ingredients))
    
        with torch.no_grad():  # Disable gradient tracking
            for batch in tqdm([unique_ingredients[i:i+batch_size]
                               for i in range(0, len(unique_ingredients), batch_size)],
                              desc="Classifying Ingredients"):
                # Process batch on GPU
                results = classifier(batch, CATEGORIES, multi_label=False)
    
                # Cache results
                for ing, result in zip(batch, results):
                    classification_cache[ing] = result['labels'][0]
    
                # Clear GPU cache
                torch.cuda.empty_cache()
    
        return classification_cache
    
    # %% Process Entire Dataset
    # Get all unique ingredients
    all_ingredients = [clean_ingredient(ing)
                       for recipe in df['NER']
                       for ing in recipe]
    unique_ingredients = list(set(all_ingredients))
    
    # Batch classify with progress bar
    _ = batch_classify(unique_ingredients, batch_size=64)
    
    # Calculate recipe costs
    df['total_cost'] = df['NER'].apply(
        lambda x: round(sum(
            MEDIAN_PRICES.get(classification_cache[clean_ingredient(ing)], 3.00)
            for ing in x
        ), 2)
    )
    
    # Dynamic price categorization
    costs = df['total_cost'].values
    
    # threshold per le categorie degli prezzi
    very_cheap, cheap, medium, expensive = np.percentile(costs, [20, 40, 60, 85])
    
    df['price_tag'] = df['total_cost'].apply(
        lambda x: 'very cheap' if x <= very_cheap
        else 'cheap' if x <= cheap
        else 'medium' if x <= medium
        else 'expensive' if x <= expensive
        else 'rich'
    )

    def get_categories(ingredient_list):
        return [classification_cache.get(ing) for ing in ingredient_list]
    
    def is_vegan(cat_list):
        return all(cat not in ['dairy', 'meat', 'seafood'] for cat in cat_list if cat is not None)
    
    def is_vegetarian(cat_list):
        return all(cat not in ['meat', 'seafood'] for cat in cat_list if cat is not None)
    
    df['categories'] = df['NER_clean'].apply(get_categories)
    df['vegan'] = df['categories'].apply(is_vegan)
    df['vegetarian'] = df['categories'].apply(is_vegetarian)
    
    # Save outputs
    df.to_csv('full_tagged_dataset_10%', index=False)

W0625 22:02:54.497000 32916 venv\Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


PyTorch version: 2.7.0+cu118
CUDA available: True
CUDA device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 3060 Ti


model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

C:\Users\akasa\PycharmProjects\FOO\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\akasa\PycharmProjects\FOO\TagManager\mvp_tagging\cache\models--MoritzLaurer--deberta-v3-base-zeroshot-v1.1-all-33. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Device set to use cuda:0
Classifying Ingredients: 100%|██████████| 707/707 [07:50<00:00,  1.50it/s]


Qui la magia finisce :)

# A partire da questo punto vengono generati i modelli di AI che vengono addestrati sul dataset a cui abbiamo lavorato.

## Nella cella sottostante vengono definite le funzioni necessarie per la creazione dei modelli

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

# Method to preprocess a dataframe
def preprocess(df, text_columns, label_column):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

# Method to train a classifier
def train_text_classifier(df, label_encoder, model_name_prefix, save_dir):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    # Save model and encoders
    model.save(f"classifiers/{save_dir}/{model_name_prefix}.h5")
    with open(f"classifiers/{save_dir}/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/{save_dir}/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# 1. Creazione del modello per la classificazione del tag 'PREPARATION_TIME'

In [13]:
# === FIRST MODEL: PREPARATION_TIME ===
df_time = df[["INGREDIENTS", "DIRECTIONS", "PREPARATION_TIME"]].copy()
df_time, prep_time_encoder = preprocess(df_time, ["INGREDIENTS", "DIRECTIONS"], "PREPARATION_TIME")
prep_time_model = train_text_classifier(df_time, prep_time_encoder, model_name_prefix="prep_time_classifier", save_dir="prep_time")

Epoch 1/10


C:\Users\akasa\PycharmProjects\FOO\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.3429 - loss: 1.5132 - val_accuracy: 0.5595 - val_loss: 1.1372
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5526 - loss: 1.1195 - val_accuracy: 0.6370 - val_loss: 0.9196
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6423 - loss: 0.9443 - val_accuracy: 0.6615 - val_loss: 0.8431
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6828 - loss: 0.8465 - val_accuracy: 0.6810 - val_loss: 0.8336
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6992 - loss: 0.7995 - val_accuracy: 0.6810 - val_loss: 0.7856
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7279 - loss: 0.7187 - val_accuracy: 0.6900 - val_loss: 0.7861
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7643 - loss: 0.6404 - val_accuracy: 0.6950 - val_loss: 0.7661
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7804 - loss: 0.6100 - val_accuracy: 0.690

Modello 'prep_time_classifier' salvato con successo.


# 2. Creazione del modello per la classificazione del tag 'DIFFICULTY'

In [14]:
# === SECOND MODEL: DIFFICULTY ===
df_difficulty = df[["INGREDIENTS", "DIRECTIONS", "DIFFICULTY"]].copy()
df_difficulty, difficulty_encoder = preprocess(df_difficulty, ["INGREDIENTS", "DIRECTIONS"], "DIFFICULTY")
difficulty_model = train_text_classifier(df_difficulty, difficulty_encoder, model_name_prefix="difficulty_classifier", save_dir="difficulty")

Epoch 1/10


C:\Users\akasa\PycharmProjects\FOO\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5245 - loss: 0.9776 - val_accuracy: 0.6760 - val_loss: 0.7345
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6629 - loss: 0.7274 - val_accuracy: 0.7015 - val_loss: 0.6459
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7178 - loss: 0.6247 - val_accuracy: 0.7265 - val_loss: 0.6062
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7526 - loss: 0.5752 - val_accuracy: 0.7300 - val_loss: 0.6010
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7847 - loss: 0.5201 - val_accuracy: 0.7325 - val_loss: 0.5907
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7979 - loss: 0.4779 - val_accuracy: 0.7305 - val_loss: 0.5904
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8000 - loss: 0.4646 - val_accuracy: 0.7260 - val_loss: 0.6106
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8239 - loss: 0.4343 - val_accuracy: 0.721

Modello 'difficulty_classifier' salvato con successo.


# 3. Creazione del modello per la classificazione del tag 'VEGAN'

In [15]:
# === THIRD MODEL: VEGAN ===
df_vegan = df[["INGREDIENTS", "DIRECTIONS", "VEGAN"]].copy()
df_vegan, vegan_encoder = preprocess(df_vegan, ["INGREDIENTS", "DIRECTIONS"], "VEGAN")
vegan_classifier_model = train_text_classifier(df_vegan, vegan_encoder, model_name_prefix="vegan_classifier", save_dir="vegan")

Epoch 1/10


C:\Users\akasa\PycharmProjects\FOO\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8274 - loss: 0.4450 - val_accuracy: 0.9115 - val_loss: 0.2174
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9216 - loss: 0.2075 - val_accuracy: 0.9445 - val_loss: 0.1620
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9518 - loss: 0.1482 - val_accuracy: 0.9485 - val_loss: 0.1560
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9576 - loss: 0.1212 - val_accuracy: 0.9475 - val_loss: 0.1638
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9666 - loss: 0.1067 - val_accuracy: 0.9450 - val_loss: 0.1691
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9699 - loss: 0.0900 - val_accuracy: 0.9460 - val_loss: 0.1733


Modello 'vegan_classifier' salvato con successo.


# 4. Creazione del modello per la classificazione del tag 'VEGETARIAN'

In [16]:
# === FOURTH MODEL: VEGETARIAN ===
df_vegetarian = df[["INGREDIENTS", "DIRECTIONS", "VEGETARIAN"]].copy()
df_vegetarian, vegetarian_encoder = preprocess(df_vegetarian, ["INGREDIENTS", "DIRECTIONS"], "VEGETARIAN")
vegetarian_classifier_model = train_text_classifier(df_vegetarian, vegetarian_encoder, model_name_prefix="vegetarian_classifier", save_dir="vegetarian")

Epoch 1/10


C:\Users\akasa\PycharmProjects\FOO\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7045 - loss: 0.5628 - val_accuracy: 0.9085 - val_loss: 0.2551
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9222 - loss: 0.2177 - val_accuracy: 0.9250 - val_loss: 0.1916
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9493 - loss: 0.1568 - val_accuracy: 0.9435 - val_loss: 0.1773
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9568 - loss: 0.1242 - val_accuracy: 0.9455 - val_loss: 0.1683
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9610 - loss: 0.1097 - val_accuracy: 0.9460 - val_loss: 0.1611
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9655 - loss: 0.0969 - val_accuracy: 0.9450 - val_loss: 0.1658
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9675 - loss: 0.0869 - val_accuracy: 0.9445 - val_loss: 0.1617
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9765 - loss: 0.0712 - val_accuracy: 0.941

Modello 'vegetarian_classifier' salvato con successo.


# 5. Creazione del modello per la classificazione del tag 'COOKING_METHOD'

In [17]:
# === FIFTH MODEL: METHOD ===
df_method = df[["INGREDIENTS", "DIRECTIONS", "COOKING_METHOD"]].copy()
df_method, method_encoder = preprocess(df_method, ["INGREDIENTS", "DIRECTIONS"], "COOKING_METHOD")
method_classifier_model = train_text_classifier(df_method, method_encoder, model_name_prefix="method_classifier", save_dir="method")

Epoch 1/10


C:\Users\akasa\PycharmProjects\FOO\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.4825 - loss: 1.6768 - val_accuracy: 0.7915 - val_loss: 0.7324
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7975 - loss: 0.6809 - val_accuracy: 0.8575 - val_loss: 0.4852
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8691 - loss: 0.4712 - val_accuracy: 0.9185 - val_loss: 0.3226
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9072 - loss: 0.3410 - val_accuracy: 0.9295 - val_loss: 0.2740
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9135 - loss: 0.3023 - val_accuracy: 0.9350 - val_loss: 0.2383
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9316 - loss: 0.2365 - val_accuracy: 0.9405 - val_loss: 0.2223
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9389 - loss: 0.2025 - val_accuracy: 0.9435 - val_loss: 0.2010
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9437 - loss: 0.1896 - val_accuracy: 0.948

Modello 'method_classifier' salvato con successo.


# 6. Creazione del modello per la classificazione del tag 'price_tag'

In [18]:
# === SIXTH MODEL: price_tag ===
df_price = df[["INGREDIENTS", "DIRECTIONS", "price_tag"]].copy()
df_price, price_encoder = preprocess(df_price, ["INGREDIENTS", "DIRECTIONS"], "price_tag")
price_model = train_text_classifier(df_price, price_encoder, model_name_prefix="price_classifier", save_dir="price")

Epoch 1/10


C:\Users\akasa\PycharmProjects\FOO\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.2725 - loss: 1.5683 - val_accuracy: 0.4725 - val_loss: 1.2728
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4566 - loss: 1.2308 - val_accuracy: 0.5520 - val_loss: 1.0548
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5216 - loss: 1.0718 - val_accuracy: 0.5810 - val_loss: 0.9784
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5517 - loss: 1.0094 - val_accuracy: 0.6035 - val_loss: 0.9434
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5909 - loss: 0.9347 - val_accuracy: 0.6115 - val_loss: 0.9134
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6096 - loss: 0.9032 - val_accuracy: 0.6090 - val_loss: 0.9042
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.6343 - loss: 0.8642 - val_accuracy: 0.6160 - val_loss: 0.8968
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6453 - loss: 0.8376 - val_accuracy: 0.620

Modello 'price_classifier' salvato con successo.


# 7. Creazione del modello per la classificazione del tag 'LACTOSE_FREE'

In [19]:
# === SEVENTH MODEL: LACTOSE_FREE ===
df_lactose = df[["INGREDIENTS", "DIRECTIONS", "LACTOSE_FREE"]].copy()
df_lactose, lactose_free_encoder = preprocess(df_lactose, ["INGREDIENTS", "DIRECTIONS"], "LACTOSE_FREE")
lactose_free_classifier_model = train_text_classifier(df_lactose, lactose_free_encoder, model_name_prefix="lactose_free_classifier", save_dir="lactose_free")

Epoch 1/10


C:\Users\akasa\PycharmProjects\FOO\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7058 - loss: 0.5418 - val_accuracy: 0.9425 - val_loss: 0.1991
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9267 - loss: 0.2255 - val_accuracy: 0.9505 - val_loss: 0.1587
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9430 - loss: 0.1768 - val_accuracy: 0.9550 - val_loss: 0.1468
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9481 - loss: 0.1571 - val_accuracy: 0.9575 - val_loss: 0.1392
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9578 - loss: 0.1280 - val_accuracy: 0.9600 - val_loss: 0.1374
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9494 - loss: 0.1416 - val_accuracy: 0.9580 - val_loss: 0.1346
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9608 - loss: 0.1205 - val_accuracy: 0.9565 - val_loss: 0.1382
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9670 - loss: 0.0970 - val_accuracy: 0.951

Modello 'lactose_free_classifier' salvato con successo.


# 8. Creazione del modello per la classificazione del tag 'GLUTEN_FREE'

In [20]:
# === EIGHTH MODEL: GLUTEN_FREE ===
df_gluten = df[["INGREDIENTS", "DIRECTIONS", "GLUTEN_FREE"]].copy()
df_gluten, gluten_free_encoder = preprocess(df_gluten, ["INGREDIENTS", "DIRECTIONS"], "GLUTEN_FREE")
gluten_free_classifier_model = train_text_classifier(df_gluten, gluten_free_encoder, model_name_prefix="gluten_free_classifier", save_dir="gluten_free")

Epoch 1/10


C:\Users\akasa\PycharmProjects\FOO\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7992 - loss: 0.5100 - val_accuracy: 0.8965 - val_loss: 0.2794
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8893 - loss: 0.2722 - val_accuracy: 0.9130 - val_loss: 0.2043
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9121 - loss: 0.2087 - val_accuracy: 0.9150 - val_loss: 0.1902
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9304 - loss: 0.1704 - val_accuracy: 0.9170 - val_loss: 0.1849
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9288 - loss: 0.1656 - val_accuracy: 0.9150 - val_loss: 0.2014
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9443 - loss: 0.1386 - val_accuracy: 0.9170 - val_loss: 0.1924
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9439 - loss: 0.1269 - val_accuracy: 0.9200 - val_loss: 0.1884


Modello 'gluten_free_classifier' salvato con successo.


## In questa cella vengono definite delle funzioni per effettuare un testing dei modelli realizzati

In [21]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # Must match training

# --- Load model and tokenizer ---
def load_model_components(model_dir, model_name_prefix):
    model = load_model(f"{model_dir}/{model_name_prefix}.h5")

    with open(f"{model_dir}/{model_name_prefix}_tokenizer.pkl", "rb") as f:
        tokenizer = pickle.load(f)

    with open(f"{model_dir}/{model_name_prefix}_label_mapping.pkl", "rb") as f:
        label_map = pickle.load(f)

    # Reverse dictionaries to decode predictions
    inv_label_map = {v: k for k, v in label_map.items()}
    return model, tokenizer, inv_label_map

def predict_and_print(recipes, model, tokenizer, inv_label_map, expected_key, prediction_label):
    # Prepare input texts for models (ingredients + instructions)
    texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]
    # Tokenize and padding
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=MAX_LEN, padding='post', truncating='post')

    # Predicting with probabilities
    preds = model.predict(padded)
    pred_classes = preds.argmax(axis=1)
    pred_labels = [inv_label_map[c] for c in pred_classes]

    # Printing results
    for i, recipe in enumerate(recipes):
        print(f"Recipe {i+1}:")
        print(f"  Expected {prediction_label}: {recipe[expected_key]}")
        print(f"  Predicted {prediction_label}: {pred_labels[i]}")
        probs_str = ", ".join([f"{inv_label_map[j]}: {preds[i][j]*100:.2f}%" for j in range(len(preds[i]))])
        print(f"  {prediction_label} probabilities: {probs_str}\n")

## 1. Testing del modello per la predizione del tempo di preparazione

In [22]:
# List of 10 recipes with expected time
prep_time_recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_time": "15 min"
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_time": "10 min"
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_time": "30 min"
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_time": "5 min"
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_time": "60 min"
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_time": "10 min"
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_time": "25 min"
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_time": "30 min"
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_time": "20 min"
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_time": "90 min"
    }
]

prep_model, prep_tokenizer, inv_prep_map = load_model_components("classifiers/prep_time", "prep_time_classifier")
predict_and_print(prep_time_recipes, prep_model, prep_tokenizer, inv_prep_map, expected_key="expected_time", prediction_label="Preparation time")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Recipe 1:
  Expected Preparation time: 15 min
  Predicted Preparation time: Fast (10-20 mins)
  Preparation time probabilities: Fast (10-20 mins): 42.55%, Medium (20-40 mins): 33.58%, Slow (40-90 mins): 4.06%, Very fast (0-10 mins): 18.01%, Very slow (90+ mins): 1.80%

Recipe 2:
  Expected Preparation time: 10 min
  Predicted Preparation time: Fast (10-20 mins)
  Preparation time probabilities: Fast (10-20 mins): 53.10%, Medium (20-40 mins): 38.93%, Slow (40-90 mins): 1.10%, Very fast (0-10 mins): 6.63%, Very slow (90+ mins): 0.25%

Recipe 3:
  Expected Preparation time: 30 min
  Predicted Preparation time: Medium (20-40 mins)
  Preparation time probabilities: Fast (10-20 mins): 17.03%, Medium (20-40 mins): 55.27%, Slow (40-90 mins): 14.93%, Very fast (0-10 mins): 9.61%, Very slow (90+ mins): 3.15%

Recipe 4:
  Expected Preparation time: 5 min
  Predicted Preparation time: Medium (20-40 mins)
  Preparation time probabilities: Fast (10-20 mins): 9.1

## 2. Testing del modello per la predizione della difficoltà

In [23]:
# List of 10 recipes with expected difficulty
difficulty_recipes = [
    {
        "ingredients": "Eggs, salt, pepper",
        "instructions": "Beat eggs with salt and pepper. Cook in a pan over medium heat until set.",
        "expected_difficulty": "Easy"
    },
    {
        "ingredients": "Chicken breast, olive oil, garlic, lemon juice, herbs",
        "instructions": "Marinate chicken with garlic, lemon juice, and herbs. Grill until cooked.",
        "expected_difficulty": "Easy"
    },
    {
        "ingredients": "Potatoes, butter, milk, salt, pepper",
        "instructions": "Boil potatoes until tender. Mash with butter and milk. Season with salt and pepper.",
        "expected_difficulty": "Easy"
    },
    {
        "ingredients": "Pasta, tomato sauce, garlic, onion, basil, olive oil",
        "instructions": "Sauté garlic and onion in olive oil. Add tomato sauce and basil. Cook pasta and mix with sauce.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "Beef stew meat, carrots, potatoes, onion, beef broth, red wine, garlic",
        "instructions": "Brown beef in a pot. Add vegetables, broth, and wine. Simmer for 2 hours until tender.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "Salmon fillets, soy sauce, ginger, garlic, honey, sesame oil",
        "instructions": "Marinate salmon with soy sauce, ginger, garlic, honey, and sesame oil. Bake at 180°C for 15 minutes.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "Duck breast, orange juice, honey, thyme, butter",
        "instructions": "Score duck skin and sear in pan. Prepare orange sauce with juice, honey, and thyme. Finish cooking duck and glaze with sauce.",
        "expected_difficulty": "Hard"
    },
    {
        "ingredients": "Lamb shank, rosemary, garlic, red wine, beef broth, carrots, onions",
        "instructions": "Brown lamb shanks. Add vegetables, broth, and wine. Slow cook in oven at 150°C for 3 hours until tender.",
        "expected_difficulty": "Hard"
    },
    {
        "ingredients": "Chocolate, eggs, sugar, butter, flour, cream, gelatin, vanilla bean",
        "instructions": "Melt chocolate and butter. Whisk eggs and sugar. Fold ingredients, bake in water bath at 160°C for 45 minutes. Cool and refrigerate overnight.",
        "expected_difficulty": "Hard"
    },
    {
        "ingredients": "Scallops, bacon, garlic, lemon, parsley, white wine",
        "instructions": "Wrap scallops with bacon and sear. Make sauce with garlic, lemon, parsley, and white wine. Serve scallops with sauce.",
        "expected_difficulty": "Hard"
    }
]

diff_model, diff_tokenizer, inv_diff_map = load_model_components("classifiers/difficulty", "difficulty_classifier")
predict_and_print(difficulty_recipes, diff_model, diff_tokenizer, inv_diff_map, expected_key="expected_difficulty", prediction_label="Difficulty")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Recipe 1:
  Expected Difficulty: Easy
  Predicted Difficulty: Easy
  Difficulty probabilities: Easy: 95.33%, Hard: 0.01%, Medium: 4.66%

Recipe 2:
  Expected Difficulty: Easy
  Predicted Difficulty: Easy
  Difficulty probabilities: Easy: 89.61%, Hard: 0.07%, Medium: 10.32%

Recipe 3:
  Expected Difficulty: Easy
  Predicted Difficulty: Easy
  Difficulty probabilities: Easy: 70.38%, Hard: 0.23%, Medium: 29.39%

Recipe 4:
  Expected Difficulty: Medium
  Predicted Difficulty: Easy
  Difficulty probabilities: Easy: 89.58%, Hard: 0.05%, Medium: 10.37%

Recipe 5:
  Expected Difficulty: Medium
  Predicted Difficulty: Medium
  Difficulty probabilities: Easy: 40.91%, Hard: 0.35%, Medium: 58.74%

Recipe 6:
  Expected Difficulty: Medium
  Predicted Difficulty: Easy
  Difficulty probabilities: Easy: 72.00%, Hard: 0.36%, Medium: 27.64%

Recipe 7:
  Expected Difficulty: Hard
  Predicted Difficulty: Easy
  Difficulty probabilities: Easy: 73.58%, Hard: 3.98%, Mediu

## 3. Testing del modello per la classificazione vegana

In [24]:
# List of 10 recipes with expected vegan
vegan_recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_vegan": True
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_vegan": False
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_vegan": False
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_vegan": False
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_vegan": False
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_vegan": True
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_vegan": False
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_vegan": False
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_vegan": False
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_vegan": True
    }
]

vegan_model, vegan_tokenizer, inv_vegan_map = load_model_components("classifiers/vegan", "vegan_classifier")
predict_and_print(vegan_recipes, vegan_model, vegan_tokenizer, inv_vegan_map, expected_key="expected_vegan", prediction_label="Vegan")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Recipe 1:
  Expected Vegan: True
  Predicted Vegan: True
  Vegan probabilities: False: 11.71%, True: 88.29%

Recipe 2:
  Expected Vegan: False
  Predicted Vegan: True
  Vegan probabilities: False: 28.38%, True: 71.62%

Recipe 3:
  Expected Vegan: False
  Predicted Vegan: False
  Vegan probabilities: False: 99.80%, True: 0.20%

Recipe 4:
  Expected Vegan: False
  Predicted Vegan: False
  Vegan probabilities: False: 99.99%, True: 0.01%

Recipe 5:
  Expected Vegan: False
  Predicted Vegan: False
  Vegan probabilities: False: 99.56%, True: 0.44%

Recipe 6:
  Expected Vegan: True
  Predicted Vegan: True
  Vegan probabilities: False: 5.80%, True: 94.20%

Recipe 7:
  Expected Vegan: False
  Predicted Vegan: False
  Vegan probabilities: False: 99.94%, True: 0.06%

Recipe 8:
  Expected Vegan: False
  Predicted Vegan: False
  Vegan probabilities: False: 99.99%, True: 0.01%

Recipe 9:
  Expected Vegan: False
  Predicted Vegan: False
  Vegan probabilities: Fal

## 4. Testing del modello per la classificazione vegetariana

In [25]:
# List of 10 recipes with expected vegetarian
vegetarian_recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_vegetarian": False  # Tuna = non vegetarian
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_vegetarian": False  # Meat sauce = non vegetarian
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_vegetarian": False  # Chicken = non vegetarian
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_vegetarian": True
    }
]

vegetarian_model, vegetarian_tokenizer, inv_vegetarian_map = load_model_components("classifiers/vegetarian", "vegetarian_classifier")
predict_and_print(vegetarian_recipes, vegetarian_model, vegetarian_tokenizer, inv_vegetarian_map, expected_key="expected_vegetarian", prediction_label="Vegetarian")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Recipe 1:
  Expected Vegetarian: True
  Predicted Vegetarian: True
  Vegetarian probabilities: False: 3.91%, True: 96.09%

Recipe 2:
  Expected Vegetarian: False
  Predicted Vegetarian: False
  Vegetarian probabilities: False: 64.04%, True: 35.96%

Recipe 3:
  Expected Vegetarian: True
  Predicted Vegetarian: True
  Vegetarian probabilities: False: 26.93%, True: 73.07%

Recipe 4:
  Expected Vegetarian: True
  Predicted Vegetarian: True
  Vegetarian probabilities: False: 0.22%, True: 99.78%

Recipe 5:
  Expected Vegetarian: False
  Predicted Vegetarian: True
  Vegetarian probabilities: False: 46.33%, True: 53.67%

Recipe 6:
  Expected Vegetarian: True
  Predicted Vegetarian: True
  Vegetarian probabilities: False: 0.02%, True: 99.98%

Recipe 7:
  Expected Vegetarian: False
  Predicted Vegetarian: False
  Vegetarian probabilities: False: 84.85%, True: 15.15%

Recipe 8:
  Expected Vegetarian: True
  Predicted Vegetarian: True
  Vegetarian probabilitie

## 5. Testing del modello per la predizione del metodo di preparazione

In [26]:
# List of 10 recipes with expected method
method_recipes = [
    {
        "ingredients": "500g potatoes, olive oil, rosemary, salt",
        "instructions": "Peel and cut the potatoes into chunks. Toss with olive oil and rosemary. Cook in oven at 200°C for 45 minutes.",
        "expected_method": "Roasted"
    },
    {
        "ingredients": "200g spaghetti, salt, water",
        "instructions": "Boil water in a large pot, add salt and cook the spaghetti until al dente.",
        "expected_method": "Boiled"
    },
    {
        "ingredients": "2 chicken breasts, olive oil, herbs, salt",
        "instructions": "Brush chicken with oil and herbs. Cook on a preheated grill for 6 minutes per side.",
        "expected_method": "Grilled"
    },
    {
        "ingredients": "4 eggs, butter, salt, pepper",
        "instructions": "Heat butter in a pan. Crack eggs in and fry until edges are crispy.",
        "expected_method": "Fried"
    },
    {
        "ingredients": "1 salmon fillet, lemon juice, herbs, salt",
        "instructions": "Place salmon in a baking dish, season, and bake at 180°C for 20 minutes.",
        "expected_method": "Baked"
    },
    {
        "ingredients": "Broccoli, carrots, zucchini, salt",
        "instructions": "Wash and cut vegetables. Eat raw or season with salt and olive oil.",
        "expected_method": "Raw"
    },
    {
        "ingredients": "Beef steak, salt, pepper, olive oil",
        "instructions": "Season steak and place under a broiler for 5–7 minutes on each side.",
        "expected_method": "Broil"
    },
    {
        "ingredients": "1 cauliflower, salt, oil, paprika",
        "instructions": "Cut cauliflower into florets. Toss with oil and paprika. Roast in oven at 220°C for 30 minutes.",
        "expected_method": "Roasted"
    },
    {
        "ingredients": "Rice, water, salt",
        "instructions": "Rinse the rice and boil in salted water for 15 minutes.",
        "expected_method": "Boiled"
    },
    {
        "ingredients": "1 can chickpeas, tahini, lemon juice, garlic, olive oil",
        "instructions": "Blend all ingredients in a food processor until smooth. Serve as is.",
        "expected_method": "Other"  # Nessuna cottura classica, solo preparazione a crudo e frullatura
    }
]

method_model, method_tokenizer, inv_method_map = load_model_components("classifiers/method", "method_classifier")
predict_and_print(method_recipes, method_model, method_tokenizer, inv_method_map, expected_key="expected_method", prediction_label="Cooking method")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Recipe 1:
  Expected Cooking method: Roasted
  Predicted Cooking method: Other
  Cooking method probabilities: Baked: 4.49%, Blended: 0.41%, Boiled: 0.39%, Broil: 2.02%, Fried: 3.48%, Grilled: 2.72%, Microwaved: 0.64%, Other: 64.84%, Pressure Cooked: 0.80%, Raw: 1.62%, Roasted: 8.93%, Slow Cooking: 3.21%, Steamed: 1.38%, Toasted: 5.06%

Recipe 2:
  Expected Cooking method: Boiled
  Predicted Cooking method: Boiled
  Cooking method probabilities: Baked: 0.15%, Blended: 0.00%, Boiled: 99.83%, Broil: 0.00%, Fried: 0.00%, Grilled: 0.00%, Microwaved: 0.00%, Other: 0.00%, Pressure Cooked: 0.00%, Raw: 0.00%, Roasted: 0.00%, Slow Cooking: 0.00%, Steamed: 0.01%, Toasted: 0.00%

Recipe 3:
  Expected Cooking method: Grilled
  Predicted Cooking method: Grilled
  Cooking method probabilities: Baked: 0.11%, Blended: 0.01%, Boiled: 0.04%, Broil: 1.37%, Fried: 2.37%, Grilled: 94.46%, Microwaved: 0.37%, Other: 0.03%, Pressure Cooked: 0.03%, Raw: 0.04%, Roasted: 0.3

## 6. Testing del modello per la classificazione del costo

In [27]:
# List of 10 recipes with expected cost category
cost_recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_cost": "very cheap"
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_cost": "cheap"
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_cost": "expensive"
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_cost": "very cheap"
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_cost": "expensive"
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_cost": "cheap"
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_cost": "medium"
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_cost": "expensive"
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_cost": "medium"
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_cost": "rich"
    }
]

cost_model, cost_tokenizer, inv_cost_map = load_model_components("classifiers/price", "price_classifier")
predict_and_print(cost_recipes, cost_model, cost_tokenizer, inv_cost_map, expected_key="expected_cost", prediction_label="Cost")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Recipe 1:
  Expected Cost: very cheap
  Predicted Cost: cheap
  Cost probabilities: cheap: 52.31%, expensive: 4.70%, medium: 21.95%, rich: 0.18%, very cheap: 20.86%

Recipe 2:
  Expected Cost: cheap
  Predicted Cost: cheap
  Cost probabilities: cheap: 51.30%, expensive: 0.85%, medium: 17.16%, rich: 0.01%, very cheap: 30.69%

Recipe 3:
  Expected Cost: expensive
  Predicted Cost: cheap
  Cost probabilities: cheap: 38.38%, expensive: 19.97%, medium: 34.68%, rich: 1.59%, very cheap: 5.37%

Recipe 4:
  Expected Cost: very cheap
  Predicted Cost: cheap
  Cost probabilities: cheap: 46.88%, expensive: 0.42%, medium: 11.50%, rich: 0.00%, very cheap: 41.19%

Recipe 5:
  Expected Cost: expensive
  Predicted Cost: cheap
  Cost probabilities: cheap: 53.53%, expensive: 2.17%, medium: 12.42%, rich: 0.07%, very cheap: 31.81%

Recipe 6:
  Expected Cost: cheap
  Predicted Cost: very cheap
  Cost probabilities: cheap: 2.10%, expensive: 0.00%, medium: 0.11%, rich: 0.

## 7. Testing del modello per la classificazione lactose free

In [28]:
# List of 10 recipes with expected lactose free
lactose_recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "lactose_free": True
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "lactose_free": True
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "lactose_free": False
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "lactose_free": False
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "lactose_free": False
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "lactose_free": True
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "lactose_free": False  # assumiamo panna non vegetale
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "lactose_free": False
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "lactose_free": False
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "lactose_free": True
    }
]

lact_model, lact_tokenizer, inv_lact_map = load_model_components("classifiers/lactose_free", "lactose_free_classifier")
predict_and_print(lactose_recipes, lact_model, lact_tokenizer, inv_lact_map, expected_key="lactose_free", prediction_label="Lactose free")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Recipe 1:
  Expected Lactose free: True
  Predicted Lactose free: True
  Lactose free probabilities: False: 9.77%, True: 90.23%

Recipe 2:
  Expected Lactose free: True
  Predicted Lactose free: True
  Lactose free probabilities: False: 1.21%, True: 98.79%

Recipe 3:
  Expected Lactose free: False
  Predicted Lactose free: False
  Lactose free probabilities: False: 100.00%, True: 0.00%

Recipe 4:
  Expected Lactose free: False
  Predicted Lactose free: False
  Lactose free probabilities: False: 99.85%, True: 0.15%

Recipe 5:
  Expected Lactose free: False
  Predicted Lactose free: False
  Lactose free probabilities: False: 99.96%, True: 0.04%

Recipe 6:
  Expected Lactose free: True
  Predicted Lactose free: True
  Lactose free probabilities: False: 1.76%, True: 98.24%

Recipe 7:
  Expected Lactose free: False
  Predicted Lactose free: False
  Lactose free probabilities: False: 98.44%, True: 1.56%

Recipe 8:
  Expected Lactose free: False
  Predict

## 8. Testing del modello per la classificazione gluten free

In [29]:
# List of 10 recipes with expected gluten free
gluten_recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "gluten_free": False
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "gluten_free": True
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "gluten_free": True
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "gluten_free": True
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "gluten_free": False
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "gluten_free": True
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "gluten_free": True
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "gluten_free": False  # ladyfingers contengono farina di frumento
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "gluten_free": False  # contiene farina
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "gluten_free": True
    }
]

gluten_model, gluten_tokenizer, inv_gluten_map = load_model_components("classifiers/gluten_free", "gluten_free_classifier")
predict_and_print(gluten_recipes, gluten_model, gluten_tokenizer, inv_gluten_map, expected_key="gluten_free", prediction_label="Gluten free")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Recipe 1:
  Expected Gluten free: False
  Predicted Gluten free: True
  Gluten free probabilities: False: 45.77%, True: 54.23%

Recipe 2:
  Expected Gluten free: True
  Predicted Gluten free: True
  Gluten free probabilities: False: 0.45%, True: 99.55%

Recipe 3:
  Expected Gluten free: True
  Predicted Gluten free: True
  Gluten free probabilities: False: 3.86%, True: 96.14%

Recipe 4:
  Expected Gluten free: True
  Predicted Gluten free: True
  Gluten free probabilities: False: 1.10%, True: 98.90%

Recipe 5:
  Expected Gluten free: False
  Predicted Gluten free: False
  Gluten free probabilities: False: 70.77%, True: 29.23%

Recipe 6:
  Expected Gluten free: True
  Predicted Gluten free: True
  Gluten free probabilities: False: 0.53%, True: 99.47%

Recipe 7:
  Expected Gluten free: True
  Predicted Gluten free: True
  Gluten free probabilities: False: 0.21%, True: 99.79%

Recipe 8:
  Expected Gluten free: False
  Predicted Gluten free: False
  Gl

In [ ]:
from llama_cpp import Llama
import pandas as pd
from tqdm import tqdm

# Inizialize model
llm = Llama(
    model_path="./phi-2.Q5_K_M.gguf",
    n_ctx=512,
    n_threads=8,
    n_gpu_layers=10,  # aumenta se usi gpu, gpt consiglia 35 per la 3060
    verbose=False
)

def generate_description(row):
    title = row["TITLE"]

    prompt = (
        f"You are a skilled food writer. Given the title of a recipe, write a short and appetizing description of the dish, highlighting its flavor, texture, and appeal. \n"
        f"Title: {title}\n"
        f"Description:"
    )

    try:
        output = llm(prompt, max_tokens=150)
        return output["choices"][0]["text"].strip()
    except Exception as e:
        return f"ERROR: {e}"


tqdm.pandas()
df["DESCRIPTION"] = df.progress_apply(generate_description, axis=1)
